In [ ]:
!pip install langchain
!pip install langchain_chroma
!pip install sentence_transformers
!pip show langchain
!pip install aiofiles
!pip install datasets
!pip install accelerate

In [ ]:
!pip install datasets

In [ ]:
!pip install aiofiles

In [1]:
import pandas as pd
df=pd.read_csv("test.csv",encoding='latin1')
df=df.head(200)
df.to_csv("test.csv")

In [2]:
df

,Unnamed: 0.1,Unnamed: 0,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (KmÃÂ²),Density (P/KmÃÂ²)
0,0,0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,1,1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,2,2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,3,3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,4,4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...
195,195,195,03bb148b87,"'I don`t like Monday, i wish it were sunday, c...",neutral,morning,46-60,Afghanistan,38928346.0,652860.0,60.0
196,196,196,b47c430fda,??? ?????? - http://bit.ly/nAcK2,neutral,noon,60-70,Albania,2877797.0,27400.0,105.0
197,197,197,0234e16010,Waiting for the last movie to finish then am t...,neutral,night,70-100,Algeria,43851044.0,2381740.0,18.0
198,198,198,be9ed19513,Happy #StarWarsDay. May the 4th be with you! ...,positive,morning,0-20,Andorra,77265.0,470.0,164.0


In [3]:
import csv
from langchain.docstore.document import Document

def read_csv_to_langchain_documents(csv_file_path):
  """
  Reads a CSV file and converts each row into a LangChain Document object.

  Args:
      csv_file_path (str): The path to the CSV file.

  Returns:
      list: A list of LangChain Document objects.
  """

  documents = []
  try:
    with open(csv_file_path, 'r', newline='') as csvfile:
      reader = csv.DictReader(csvfile)

      for row in reader:
        # Construct page content (adjust column names and processing as needed)
        content = row["text"]

                # Create metadata using all other columns
        metadata = {key: value for key, value in row.items() if key != "text"}

        documents.append(Document(page_content=content, metadata=metadata))

  except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path}' not found.")
  except Exception as e:
    print(f"An error occurred while processing the CSV file: {str(e)}")

  return documents



In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter



documents = read_csv_to_langchain_documents("test.csv")


embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")

db = Chroma.from_documents(documents, embedding_function)



In [ ]:

query = "yo"
docs = db.similarity_search(query)
print("docs")
filter_metadata = {"sentiment": "negative"}
filtered_documents = db.filter(where=filter_metadata)

print("Page Content:")
print(filtered_documents[0].page_content)
print("Meta Data:")
print(filtered_documents[0].metadata)

In [61]:
from typing import Optional

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator
from langchain_core.pydantic_v1 import BaseModel







class Search(BaseModel):
    query: str
    sentiment:str
    country: str

search_query = Search(query="RAG", sentiment="positive", country="Afghanistan")


def construct_comparisons(query: Search):
    comparisons = []
    if query.sentiment is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GT,
                attribute="sentiment",
                value=query.sentiment,
            )
        )
    if query.country is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="country",
                value=query.country,
            )
        )
    return comparisons


comparisons = construct_comparisons(search_query)
_filter = Operation(operator=Operator.AND, arguments=comparisons)

ChromaTranslator().visit_operation(_filter)




{'$and': [{'sentiment': {'$gt': 'positive'}},
  {'country': {'$eq': 'Afghanistan'}}]}

In [ ]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")

sentences = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          ]

embeddings = model.encode(sentences)


In [ ]:
output_dir = "/microsoftLLM"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

saved_model_dir = "./LLM"
model = AutoModelForCausalLM.from_pretrained(saved_model_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)